In [1]:
import pandas as pd
import random

import geopandas as gpd 
from shapely.geometry import Point, Polygon
import h3

import plotly.express as px

import json

In [17]:
# Cargar datos geoespaciales en un DataFrame GeoPandas
gdf = gpd.read_file('data/df_italia.geojson')
gdf['geoid'] = gdf.index.astype(str)
gdf.geometry = gdf.geometry.to_crs(epsg = 4326) # 4326    # 3857
gdf.head()

,COD_UTS,DEN_UTS,h3,count,geometry,geoid
0,215,Milano,871f98241ffffff,20,"POLYGON ((8.86056 45.35089, 8.86204 45.33849, ...",0
1,215,Milano,871f98245ffffff,18,"POLYGON ((8.84271 45.36792, 8.84419 45.35552, ...",1
2,215,Milano,871f98248ffffff,30,"POLYGON ((8.90969 45.33698, 8.91115 45.32458, ...",2
3,215,Milano,871f98249ffffff,33,"POLYGON ((8.94097 45.34010, 8.94244 45.32769, ...",3
4,215,Milano,871f9824affffff,3,"POLYGON ((8.89625 45.31681, 8.89772 45.30440, ...",4


In [18]:
type(gdf)

geopandas.geodataframe.GeoDataFrame

In [44]:
fig = px.choropleth_mapbox(gdf,
                           geojson=gdf.__geo_interface__,#['geometry']
                           locations=gdf.geoid,
                           color='count',
                           hover_data=["DEN_UTS"],
                           labels={'count':'Unidades'},
                           featureidkey='properties.geoid',
                           center={'lat': 45.5, 'lon':9.5},
                           mapbox_style='carto-positron',
                           zoom=10,
                           opacity=0.1,
                           )

# darle formato al toltip de los poligonos
# fig.update_traces(hovertemplate='<b>%{location}</b><br>Valor: %{z}')










# Agregar evento de clic
fig.update_layout(
    clickmode='event+select',
    showlegend=False
)

def handle_click(trace, points, selector):
    if points.point_inds:
        selected_polygon = points.point_inds[0]
        print(f"Polígono seleccionado: {selected_polygon}")

fig.data[0].on_click(handle_click)





# Agrandar y mover labels

fig.update_layout(margin={'l': 0, 't': 0, 'r': 0, 'b': 0})
fig.update_coloraxes(
    colorbar=dict(
        lenmode="fraction",
        len=0.4,
        x=0,
        y=0.25
    )
)


fig.show()

In [36]:
fig = px.choropleth_mapbox(gdf,
                           geojson=gdf.__geo_interface__,#['geometry']
                           locations=gdf.geoid,
                           color='count',
                           hover_data=["DEN_UTS"],
                           labels={'count':'Unidades'},
                           featureidkey='properties.geoid',
                           center={'lat': 45.5, 'lon':9.5},
                           mapbox_style='carto-positron',
                           zoom=6)

# darle formato al toltip de los poligonos
# fig.update_traces(hovertemplate='<b>%{location}</b><br>Valor: %{z}')










# Agregar evento de clic
fig.update_layout(
    clickmode='event+select',
    showlegend=False
)

selected_polygons = []

def handle_click(trace, points, selector):
    if points.point_inds:
        selected_polygon = points.point_inds[0]
        if selected_polygon not in selected_polygons:
            selected_polygons.append(selected_polygon)
        else:
            selected_polygons.remove(selected_polygon)
        print(f"Polígonos seleccionados: {selected_polygons}")

fig.data[0].on_click(handle_click)





# Agrandar y mover labels

fig.update_layout(margin={'l': 0, 't': 0, 'r': 0, 'b': 0})
fig.update_coloraxes(
    colorbar=dict(
        lenmode="fraction",
        len=0.4,
        x=0,
        y=0.25
    )
)


fig.show()

In [5]:
import cbsodata

In [6]:
geodata_url = 'https://cartomap.github.io/nl/wgs84/gemeente_2023.geojson'
municipal_boundaries = gpd.read_file(geodata_url)
municipal_boundaries.head()

,id,statcode,jrstatcode,statnaam,rubriek,FID,geometry
0,1,GM0014,2023GM0014,Groningen,gemeente,gemeente_gegeneraliseerd.cd758af1-98f2-41e5-b5...,"POLYGON ((6.77300 53.28300, 6.69800 53.25600, ..."
1,2,GM0034,2023GM0034,Almere,gemeente,gemeente_gegeneraliseerd.0f8c7e25-8a8e-4cb8-a5...,"POLYGON ((5.35100 52.40000, 5.30100 52.38200, ..."
2,3,GM0037,2023GM0037,Stadskanaal,gemeente,gemeente_gegeneraliseerd.22dc258d-40e1-4a28-9e...,"POLYGON ((7.01500 53.07200, 7.03400 53.06500, ..."
3,4,GM0047,2023GM0047,Veendam,gemeente,gemeente_gegeneraliseerd.584fa43a-9541-49c9-a4...,"POLYGON ((6.96200 53.11000, 6.96100 53.09600, ..."
4,5,GM0050,2023GM0050,Zeewolde,gemeente,gemeente_gegeneraliseerd.daefbcb4-a795-4c3b-a8...,"POLYGON ((5.58900 52.41100, 5.62300 52.40900, ..."


In [7]:
data = pd.DataFrame(cbsodata.get_data('83765NED', select = ['WijkenEnBuurten', 'Codering_3', 'GeboorteRelatief_25']))
data['Codering_3'] = data['Codering_3'].str.strip()
data.head()

,WijkenEnBuurten,Codering_3,GeboorteRelatief_25
0,Nederland,NL00,9
1,Aa en Hunze,GM1680,6
2,Wijk 00 Annen,WK168000,9
3,Annen,BU16800000,10
4,Verspreide huizen Annen,BU16800009,0


In [8]:
# Link data from Statistics Netherlands to geodata
municipal_boundaries = pd.merge(municipal_boundaries, data,
                               left_on = "statcode", 
                               right_on = "Codering_3")
municipal_boundaries.head()

,id,statcode,jrstatcode,statnaam,rubriek,FID,geometry,WijkenEnBuurten,Codering_3,GeboorteRelatief_25
0,1,GM0014,2023GM0014,Groningen,gemeente,gemeente_gegeneraliseerd.cd758af1-98f2-41e5-b5...,"POLYGON ((6.77300 53.28300, 6.69800 53.25600, ...",Groningen,GM0014,9
1,2,GM0034,2023GM0034,Almere,gemeente,gemeente_gegeneraliseerd.0f8c7e25-8a8e-4cb8-a5...,"POLYGON ((5.35100 52.40000, 5.30100 52.38200, ...",Almere,GM0034,11
2,3,GM0037,2023GM0037,Stadskanaal,gemeente,gemeente_gegeneraliseerd.22dc258d-40e1-4a28-9e...,"POLYGON ((7.01500 53.07200, 7.03400 53.06500, ...",Stadskanaal,GM0037,8
3,4,GM0047,2023GM0047,Veendam,gemeente,gemeente_gegeneraliseerd.584fa43a-9541-49c9-a4...,"POLYGON ((6.96200 53.11000, 6.96100 53.09600, ...",Veendam,GM0047,7
4,5,GM0050,2023GM0050,Zeewolde,gemeente,gemeente_gegeneraliseerd.daefbcb4-a795-4c3b-a8...,"POLYGON ((5.58900 52.41100, 5.62300 52.40900, ...",Zeewolde,GM0050,9


In [9]:
#CRS: EPSG 3857 (web mercator projection wgs84)
municipal_boundaries.crs
# municipal_boundaries = municipal_boundaries.to_crs(epsg=3857)
municipal_boundaries = municipal_boundaries.to_crs(epsg=4326)

In [10]:
#First column: Geoid, geometry column and  data columns
gdf_choro = municipal_boundaries.copy()
gdf_choro['geoid'] = gdf_choro.index.astype(str)
gdf_choro = gdf_choro[['geoid', 'geometry', 'statnaam', 'GeboorteRelatief_25']]
gdf_choro.head(3)

,geoid,geometry,statnaam,GeboorteRelatief_25
0,0,"POLYGON ((6.77300 53.28300, 6.69800 53.25600, ...",Groningen,9
1,1,"POLYGON ((5.35100 52.40000, 5.30100 52.38200, ...",Almere,11
2,2,"POLYGON ((7.01500 53.07200, 7.03400 53.06500, ...",Stadskanaal,8


In [11]:
type(gdf_choro)

geopandas.geodataframe.GeoDataFrame

In [12]:
fig = px.choropleth_mapbox(gdf_choro,
                           geojson=gdf_choro.__geo_interface__,#['geometry']
                           locations=gdf_choro.geoid,
                           color='GeboorteRelatief_25',
                           featureidkey='properties.geoid',
                           center={'lat': 52.213, 'lon':5.2794},
                           mapbox_style='carto-positron',
                           zoom=6)
fig.show()

In [13]:
fig = px.choropleth_mapbox(gdf_choro,
                           geojson=gdf_choro.__geo_interface__,#['geometry']
                           locations=gdf_choro.geoid,
                           color='GeboorteRelatief_25',
                           featureidkey='properties.geoid',
                           center={'lat': 52.213, 'lon':5.2794},
                           mapbox_style='carto-positron',
                           zoom=6)

# Agregar evento de clic
fig.update_layout(
    clickmode='event+select',
    showlegend=False
)

def handle_click(trace, points, selector):
    if points.point_inds:
        selected_polygon = points.point_inds[0]
        print(f"Polígono seleccionado: {selected_polygon}")

fig.data[0].on_click(handle_click)

fig.show()

In [14]:
with open('data/df_italia.geojson', 'r') as infile:
    geo_json = json.load(infile)